<a href="https://colab.research.google.com/github/jonitorta/Ejercicios_Machine_learning./blob/main/Housing_pred_neural_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### House price prediction with neural net


In [1]:
import sys 
import numpy as np
import pandas as pd

from tensorflow import keras
from keras import layers

print(f"python version {sys.version}")
print(f"numpy version {np.__version__}")
print(f"pandas version {pd.__version__}")

python version 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
numpy version 1.21.6
pandas version 1.3.5


In [2]:
from tensorflow.keras.datasets import boston_housing

In [3]:
(train_data, train_targets), (test_data, test_targets) = (boston_housing.load_data() )

65536/57026 [==================================] - 0s 0us/step


In [4]:
print(f"train dataset shape is {train_data.shape}")
print(f"test dataset shape is {test_data.shape}")

train dataset shape is (404, 13)
test dataset shape is (102, 13)


In [5]:
#Let's perform standarization
mean = train_data.mean(axis = 0) 
#substract mean from each row 
train_data -=mean
#compute the std
std = train_data.std(axis = 0)
#divide with the std
train_data /= std
#Do same for test data
test_data -=mean
test_data /=std

In [6]:
def build_model():
  """
  Create a neural net of 3 dense layers(all layers are conected), the optimizer is rmsprop that is
  a variation of gradient desent with variational learning rate, loss function is mean square error
  because is a good one in regression models.
  """
  model = keras.Sequential([
                layers.Dense(64, activation = "relu"),
                layers.Dense(64, activation = "relu"),
                layers.Dense(1)
  ])
  model.compile(optimizer = "rmsprop", loss = "mse", metrics = ["mae"] )
  return model

In [7]:
#Here I implement a k-fold validation
#Number of folds
k = 3
#Divide how many samples per fold
num_val_samples = len(train_data)//k
num_epoch = 1000
all_scores = []
for i in range(k):
  print(f"Processing fold #{i}")
  #Take a slice from train_data and train_targets
  val_data = train_data[i*num_val_samples:(i+1)*num_val_samples]
  val_targets = train_targets[i*num_val_samples:(i+1)*num_val_samples]
  #Tale the others k-1 folds and concatenate them
  partial_train_data = np.concatenate(
      [train_data[:i*num_val_samples],
       train_data[(i+1)*num_val_samples:]], axis = 0
  )
  partial_train_samples = np.concatenate(
      [train_targets[:i*num_val_samples],
       train_targets[(i+1)*num_val_samples:]], axis = 0
  )
  #Create the model
  model = build_model()
  #Fit the model with the partial data
  model.fit(partial_train_data, partial_train_samples,
            epochs = num_epoch, batch_size = 16, verbose = 0 )
  #Evaluate the model with the val data
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose = 0)
  #Save the mae
  all_scores.append(val_mae)

Processing fold #0
Processing fold #1
Processing fold #2


In [11]:
#The average error is $2700
np.mean(all_scores)

2.765585025151571

In [12]:
predictions = model.predict(test_data)